In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import warnings

In [2]:
# Suppress UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

In [19]:
df_trips = pd.read_csv('trips2020_1.5M.csv.zip')
df_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,01/01/2020 02:50:43 AM,01/01/2020 02:53:10 AM,1.0,0.80,1.0,N,141,229,1.0,4.5,3.0,0.5,1.00,0.0,0.3,9.30,2.5
1,1.0,02/24/2020 10:38:12 AM,02/24/2020 11:09:11 AM,1.0,5.60,1.0,N,100,87,2.0,23.5,2.5,0.5,0.00,0.0,0.3,26.80,2.5
2,2.0,01/18/2020 11:52:03 PM,01/19/2020 11:25:26 PM,1.0,5.04,1.0,N,230,7,1.0,18.0,0.5,0.5,4.36,0.0,0.3,26.16,2.5
3,2.0,02/02/2020 11:14:26 PM,02/02/2020 11:26:56 PM,1.0,1.88,1.0,N,255,17,1.0,10.0,0.5,0.5,2.26,0.0,0.3,13.56,0.0
4,2.0,03/11/2020 07:03:20 PM,03/11/2020 07:18:51 PM,2.0,2.32,1.0,N,264,264,1.0,11.5,1.0,0.5,1.00,0.0,0.3,16.80,2.5


# Data cleaning

## Adding the trip distance length in time units

In [20]:
# To create the duration cost matrix later, we will need to use the trip distances in time units, which we currently do not have
# We can do this based on the two datetime columns

# First, turning the two columns into datetime objects
df_trips["tpep_pickup_datetime"] = pd.to_datetime(df_trips["tpep_pickup_datetime"])
df_trips["tpep_dropoff_datetime"] = pd.to_datetime(df_trips["tpep_dropoff_datetime"])

# Then, we can calculate the time length of the trip
df_trips['time_length'] = (df_trips["tpep_dropoff_datetime"] - df_trips["tpep_pickup_datetime"]).dt.total_seconds() / 3600

df_trips

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,time_length
0,1.0,2020-01-01 02:50:43,2020-01-01 02:53:10,1.0,0.80,1.0,N,141,229,1.0,4.5,3.0,0.5,1.00,0.0,0.3,9.30,2.5,0.040833
1,1.0,2020-02-24 10:38:12,2020-02-24 11:09:11,1.0,5.60,1.0,N,100,87,2.0,23.5,2.5,0.5,0.00,0.0,0.3,26.80,2.5,0.516389
2,2.0,2020-01-18 23:52:03,2020-01-19 23:25:26,1.0,5.04,1.0,N,230,7,1.0,18.0,0.5,0.5,4.36,0.0,0.3,26.16,2.5,23.556389
3,2.0,2020-02-02 23:14:26,2020-02-02 23:26:56,1.0,1.88,1.0,N,255,17,1.0,10.0,0.5,0.5,2.26,0.0,0.3,13.56,0.0,0.208333
4,2.0,2020-03-11 19:03:20,2020-03-11 19:18:51,2.0,2.32,1.0,N,264,264,1.0,11.5,1.0,0.5,1.00,0.0,0.3,16.80,2.5,0.258611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499995,2.0,2020-06-24 11:37:15,2020-06-24 11:47:06,2.0,1.66,1.0,N,229,263,1.0,8.5,0.0,0.5,2.36,0.0,0.3,14.16,2.5,0.164167
1499996,2.0,2020-03-12 19:25:45,2020-03-12 19:43:49,2.0,7.19,1.0,N,132,135,2.0,21.5,1.0,0.5,0.00,0.0,0.3,23.30,0.0,0.301111
1499997,2.0,2020-02-26 19:40:11,2020-02-26 19:44:56,1.0,0.66,1.0,N,68,90,1.0,5.0,1.0,0.5,1.00,0.0,0.3,10.30,2.5,0.079167
1499998,1.0,2020-02-16 12:39:08,2020-02-16 12:45:27,2.0,0.80,1.0,N,170,233,1.0,6.0,2.5,0.5,0.00,0.0,0.3,9.30,2.5,0.105278


## Dropping useless columns

In [21]:
df_trips = df_trips.drop(columns=['tpep_dropoff_datetime', 'tpep_pickup_datetime', 'store_and_fwd_flag', 'VendorID', 'RatecodeID', 'passenger_count', 'payment_type'])
df_trips

,trip_distance,PULocationID,DOLocationID,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,time_length
0,0.80,141,229,4.5,3.0,0.5,1.00,0.0,0.3,9.30,2.5,0.040833
1,5.60,100,87,23.5,2.5,0.5,0.00,0.0,0.3,26.80,2.5,0.516389
2,5.04,230,7,18.0,0.5,0.5,4.36,0.0,0.3,26.16,2.5,23.556389
3,1.88,255,17,10.0,0.5,0.5,2.26,0.0,0.3,13.56,0.0,0.208333
4,2.32,264,264,11.5,1.0,0.5,1.00,0.0,0.3,16.80,2.5,0.258611
...,...,...,...,...,...,...,...,...,...,...,...,...
1499995,1.66,229,263,8.5,0.0,0.5,2.36,0.0,0.3,14.16,2.5,0.164167
1499996,7.19,132,135,21.5,1.0,0.5,0.00,0.0,0.3,23.30,0.0,0.301111
1499997,0.66,68,90,5.0,1.0,0.5,1.00,0.0,0.3,10.30,2.5,0.079167
1499998,0.80,170,233,6.0,2.5,0.5,0.00,0.0,0.3,9.30,2.5,0.105278


## Limiting the variables' values

In [22]:
# To clean the data, we will remove rows with missing values and outliers by filtering the data on specific conditions (explained in the report)
# Manually checking the data for DOLocationID and PULocationID using Excel's filter feature, we can see that there are no missing values or outliers in these columns
print(f"Dataframe shape before cleaning: {df_trips.shape}")

df_trips_filtered = df_trips.loc[
    (df_trips['trip_distance'] < 21.0) & 
    (df_trips['trip_distance'] > 0.2) & 
    (df_trips['time_length'] > 0.02)
]

print(f"Dataframe shape after cleaning: {df_trips_filtered.shape}")

Dataframe shape before cleaning: (1500000, 12)
Dataframe shape after cleaning: (1461949, 12)


# Memory usage optimization

In [23]:
df_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trip_distance          1500000 non-null  float64
 1   PULocationID           1500000 non-null  int64  
 2   DOLocationID           1500000 non-null  int64  
 3   fare_amount            1500000 non-null  float64
 4   extra                  1500000 non-null  float64
 5   mta_tax                1500000 non-null  float64
 6   tip_amount             1500000 non-null  float64
 7   tolls_amount           1500000 non-null  float64
 8   improvement_surcharge  1500000 non-null  float64
 9   total_amount           1500000 non-null  float64
 10  congestion_surcharge   1500000 non-null  float64
 11  time_length            1500000 non-null  float64
dtypes: float64(10), int64(2)
memory usage: 137.3 MB


In [24]:
# To reduce memory usage, we will limit the datatypes of the columns
df_trips[['fare_amount', 'extra', 'mta_tax', 'tip_amount','tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'trip_distance']] = df_trips[['fare_amount', 'extra', 'mta_tax', 'tip_amount','tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'trip_distance']].astype('float16')
df_trips['total_amount'] = df_trips['total_amount'].astype('float32')
df_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trip_distance          1500000 non-null  float16
 1   PULocationID           1500000 non-null  int64  
 2   DOLocationID           1500000 non-null  int64  
 3   fare_amount            1500000 non-null  float16
 4   extra                  1500000 non-null  float16
 5   mta_tax                1500000 non-null  float16
 6   tip_amount             1500000 non-null  float16
 7   tolls_amount           1500000 non-null  float16
 8   improvement_surcharge  1500000 non-null  float16
 9   total_amount           1500000 non-null  float32
 10  congestion_surcharge   1500000 non-null  float16
 11  time_length            1500000 non-null  float64
dtypes: float16(8), float32(1), float64(1), int64(2)
memory usage: 62.9 MB


c:\Users\Damo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:134: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


# Data splitting

In [26]:
inputs = df_trips_filtered[['PULocationID', 'DOLocationID']]
labelDistance = df_trips_filtered['trip_distance']

x_trainDistance, x_testDistance, y_trainDistance, y_testDistance = train_test_split(inputs, labelDistance, random_state=69)

# Predictive model - distance cost matrix

## Final model training

In [27]:
# Train chosen model
finModelDistance = RandomForestRegressor()
finModelDistance.fit(x_trainDistance, y_trainDistance)

RandomForestRegressor()

## Predictions

### Importing depot and shop coordinates

In [36]:
# Depot coordinates and DF
df_depot = pd.DataFrame({'id':'Depot', 'location_id':112}, index=[0])

# Shops' coordinates and DF
df_shops = pd.read_csv('2020_shop_locations.csv')
df_shops = df_shops.drop(columns=['Unnamed: 0', 'demand(kg)', 'stage', 'lat', 'long', 'borought', 'zone'])

# Create new df that concatenates depot coordinates with df_shops
df_points = pd.concat([df_depot, df_shops], ignore_index=True)
df_points

,id,location_id
0,Depot,112.0
1,ChIJwTC7A55ZwokRPNX9g7ngbaI,79.0
2,ChIJU1XImaNZwokRutunetC8XeE,234.0
3,ChIJG7L-TLVbwokRT36uIrwz2Mo,65.0
4,ChIJ5cPkuBtgwokRn55JgpGqjFA,92.0
...,...,...
216,ChIJ3wsoxKFZwokRRg8OvEjE0bc,107.0
217,ChIJQRpR6Jj0wokRl9myJ19eEvU,242.0
218,ChIJQYmJtx5awokR45_sopkXVNE,231.0
219,ChIJh4MG8h9awokR1kDLslu2C3E,231.0


### Distance cost matrix

In [37]:
# Predict the cost matrix using every couple of ids in df_points
## First create a new df for the cost matrix
costMatrixDistance = pd.DataFrame(columns=df_points['id'], index=df_points['id'])

## Second fill the cost matrix with the predicted values
for id1 in df_points['id']:
    # Get the latitude and longitude for id1
    locationID1 = df_points.loc[df_points['id'] == id1, 'location_id'].values[0]
    
    for id2 in df_points['id']:
        # Get the latitude and longitude for id2
        locationID2 = df_points.loc[df_points['id'] == id2, 'location_id'].values[0]

        costMatrixDistance.loc[id1, id2] = finModelDistance.predict([[locationID1, locationID2]])[0]

costMatrixDistance

id,Depot,ChIJwTC7A55ZwokRPNX9g7ngbaI,ChIJU1XImaNZwokRutunetC8XeE,ChIJG7L-TLVbwokRT36uIrwz2Mo,ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ24V7r_31wokR-S71l2zqrwc,ChIJiUJ1DI5ZwokRWdK6SPg9BOY,ChIJYTxm0Gf2wokR5V0iFDQ-2x0,ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJYzaRC2REwokRaH2rHpflSYk,...,ChIJReQHww5FwokRGlFWAkn4opg,ChIJQwtmq3v2wokRNOjx5cEBhXg,ChIJt2vTu1lawokRfVV3pBzItgU,ChIJX9DK2EBEwokRM9kDwjBVtag,ChIJmZKM98dgwokRZ6ZvctE2ImQ,ChIJ3wsoxKFZwokRRg8OvEjE0bc,ChIJQRpR6Jj0wokRl9myJ19eEvU,ChIJQYmJtx5awokR45_sopkXVNE,ChIJh4MG8h9awokR1kDLslu2C3E,ChIJbdnGmlxewokRobPWHJXQVjM
id,,,,,,,,,,,,,,,,,,,,,
Depot,0.908164,3.624313,3.06439,4.389933,10.402506,7.179099,0.970495,5.890938,17.795796,17.759441,...,13.875805,8.577781,5.44481,15.372453,11.601055,2.636302,13.965268,6.172048,6.172048,6.126072
ChIJwTC7A55ZwokRPNX9g7ngbaI,4.487928,0.69232,1.275166,3.404734,11.91588,6.139603,1.165866,6.603732,17.565902,17.655366,...,11.143219,8.364849,5.315607,15.963847,11.432615,0.907903,11.489765,1.901283,1.901283,7.48685
ChIJU1XImaNZwokRutunetC8XeE,5.226573,1.238316,0.69339,4.251927,12.064494,5.551015,1.546753,5.13778,17.167576,18.68499,...,13.796589,7.155312,5.823538,12.936463,11.820069,0.758407,14.100627,2.324565,2.324565,8.534553
ChIJG7L-TLVbwokRT36uIrwz2Mo,4.35525,2.95688,4.039189,0.696478,9.573396,10.271436,2.541625,11.223,16.794486,4.019016,...,8.569997,11.704933,2.211792,11.801273,12.724804,4.125083,14.666276,2.171729,2.171729,6.119356
ChIJ5cPkuBtgwokRn55JgpGqjFA,13.477798,12.874358,10.590967,9.80648,0.996817,10.636402,11.766712,11.223698,9.554601,18.6048,...,6.510232,11.005756,6.830633,6.263304,8.757591,13.914388,12.551039,13.131675,13.131675,12.911588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJ3wsoxKFZwokRRg8OvEjE0bc,4.79565,0.923201,0.723722,4.286476,11.202902,5.612903,1.651204,5.459457,16.934039,17.158424,...,11.603017,7.469938,6.504683,14.693145,12.141539,0.67782,13.880918,2.55509,2.55509,8.124578
ChIJQRpR6Jj0wokRl9myJ19eEvU,13.462167,7.422576,10.747481,11.828408,7.453325,7.339817,5.455323,5.742348,20.610216,15.2434,...,12.606605,4.035693,9.354276,13.006796,13.507891,11.292802,1.349578,11.045547,11.045547,6.673253
ChIJQYmJtx5awokR45_sopkXVNE,5.905736,2.135587,2.268411,2.207256,13.13874,9.151589,0.889933,6.893867,18.390186,18.863063,...,10.674825,9.968347,3.503519,12.979222,12.969834,2.693479,13.744055,0.726178,0.726178,9.72825


In [38]:
# Export the DataFrame to a CSV file
costMatrixDistance.to_csv('costMatrixDistance.csv')

# Predictive model - duration cost matrix

## Data splitting

In [39]:
labelTime = df_trips_filtered['time_length']

x_trainTime, x_testTime, y_trainTime, y_testTime = train_test_split(inputs, labelTime, random_state=420)

## Final model training

In [40]:
# Train chosen model
finModelDuration = RandomForestRegressor()
finModelDuration.fit(x_trainTime, y_trainTime)

RandomForestRegressor()

## Predictions

### Duration cost matrix

In [41]:
# Predict the cost matrix using every couple of ids in df_points
## First create a new df for the cost matrix
costMatrixDuration = pd.DataFrame(columns=df_points['id'], index=df_points['id'])

## Second fill the cost matrix with the predicted values
for id1 in df_points['id']:
    # Get the latitude and longitude for id1
    locationID1 = df_points.loc[df_points['id'] == id1, 'location_id'].values[0]
    
    for id2 in df_points['id']:
        # Get the latitude and longitude for id2
        locationID2 = df_points.loc[df_points['id'] == id2, 'location_id'].values[0]

        costMatrixDuration.loc[id1, id2] = finModelDistance.predict([[locationID1, locationID2]])[0]

costMatrixDuration

id,Depot,ChIJwTC7A55ZwokRPNX9g7ngbaI,ChIJU1XImaNZwokRutunetC8XeE,ChIJG7L-TLVbwokRT36uIrwz2Mo,ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ24V7r_31wokR-S71l2zqrwc,ChIJiUJ1DI5ZwokRWdK6SPg9BOY,ChIJYTxm0Gf2wokR5V0iFDQ-2x0,ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJYzaRC2REwokRaH2rHpflSYk,...,ChIJReQHww5FwokRGlFWAkn4opg,ChIJQwtmq3v2wokRNOjx5cEBhXg,ChIJt2vTu1lawokRfVV3pBzItgU,ChIJX9DK2EBEwokRM9kDwjBVtag,ChIJmZKM98dgwokRZ6ZvctE2ImQ,ChIJ3wsoxKFZwokRRg8OvEjE0bc,ChIJQRpR6Jj0wokRl9myJ19eEvU,ChIJQYmJtx5awokR45_sopkXVNE,ChIJh4MG8h9awokR1kDLslu2C3E,ChIJbdnGmlxewokRobPWHJXQVjM
id,,,,,,,,,,,,,,,,,,,,,
Depot,0.908164,3.624313,3.06439,4.389933,10.402506,7.179099,0.970495,5.890938,17.795796,17.759441,...,13.875805,8.577781,5.44481,15.372453,11.601055,2.636302,13.965268,6.172048,6.172048,6.126072
ChIJwTC7A55ZwokRPNX9g7ngbaI,4.487928,0.69232,1.275166,3.404734,11.91588,6.139603,1.165866,6.603732,17.565902,17.655366,...,11.143219,8.364849,5.315607,15.963847,11.432615,0.907903,11.489765,1.901283,1.901283,7.48685
ChIJU1XImaNZwokRutunetC8XeE,5.226573,1.238316,0.69339,4.251927,12.064494,5.551015,1.546753,5.13778,17.167576,18.68499,...,13.796589,7.155312,5.823538,12.936463,11.820069,0.758407,14.100627,2.324565,2.324565,8.534553
ChIJG7L-TLVbwokRT36uIrwz2Mo,4.35525,2.95688,4.039189,0.696478,9.573396,10.271436,2.541625,11.223,16.794486,4.019016,...,8.569997,11.704933,2.211792,11.801273,12.724804,4.125083,14.666276,2.171729,2.171729,6.119356
ChIJ5cPkuBtgwokRn55JgpGqjFA,13.477798,12.874358,10.590967,9.80648,0.996817,10.636402,11.766712,11.223698,9.554601,18.6048,...,6.510232,11.005756,6.830633,6.263304,8.757591,13.914388,12.551039,13.131675,13.131675,12.911588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJ3wsoxKFZwokRRg8OvEjE0bc,4.79565,0.923201,0.723722,4.286476,11.202902,5.612903,1.651204,5.459457,16.934039,17.158424,...,11.603017,7.469938,6.504683,14.693145,12.141539,0.67782,13.880918,2.55509,2.55509,8.124578
ChIJQRpR6Jj0wokRl9myJ19eEvU,13.462167,7.422576,10.747481,11.828408,7.453325,7.339817,5.455323,5.742348,20.610216,15.2434,...,12.606605,4.035693,9.354276,13.006796,13.507891,11.292802,1.349578,11.045547,11.045547,6.673253
ChIJQYmJtx5awokR45_sopkXVNE,5.905736,2.135587,2.268411,2.207256,13.13874,9.151589,0.889933,6.893867,18.390186,18.863063,...,10.674825,9.968347,3.503519,12.979222,12.969834,2.693479,13.744055,0.726178,0.726178,9.72825


In [42]:
# Export the DataFrame to a CSV file
costMatrixDuration.to_csv('costMatrixDuration.csv')

# Predictive model - financial cost matrix

## Data splitting

In [43]:
labelFinancial = df_trips_filtered['total_amount'] - df_trips_filtered['tip_amount']

x_trainFinancial, x_testFinancial, y_trainFinancial, y_testFinancial = train_test_split(inputs, labelFinancial, random_state=420)

## Final model training

In [44]:
# Train chosen model
finModelFinancial = RandomForestRegressor()
finModelFinancial.fit(x_trainFinancial, y_trainFinancial)

RandomForestRegressor()

## Predictions

### Financial cost matrix

In [45]:
# Predict the cost matrix using every couple of ids in df_points
## First create a new df for the cost matrix
costMatrixFinancial = pd.DataFrame(columns=df_points['id'], index=df_points['id'])

## Second fill the cost matrix with the predicted values
for id1 in df_points['id']:
    # Get the latitude and longitude for id1
    locationID1 = df_points.loc[df_points['id'] == id1, 'location_id'].values[0]
    
    for id2 in df_points['id']:
        # Get the latitude and longitude for id2
        locationID2 = df_points.loc[df_points['id'] == id2, 'location_id'].values[0]

        costMatrixFinancial.loc[id1, id2] = finModelDistance.predict([[locationID1, locationID2]])[0]

costMatrixFinancial

id,Depot,ChIJwTC7A55ZwokRPNX9g7ngbaI,ChIJU1XImaNZwokRutunetC8XeE,ChIJG7L-TLVbwokRT36uIrwz2Mo,ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ24V7r_31wokR-S71l2zqrwc,ChIJiUJ1DI5ZwokRWdK6SPg9BOY,ChIJYTxm0Gf2wokR5V0iFDQ-2x0,ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJYzaRC2REwokRaH2rHpflSYk,...,ChIJReQHww5FwokRGlFWAkn4opg,ChIJQwtmq3v2wokRNOjx5cEBhXg,ChIJt2vTu1lawokRfVV3pBzItgU,ChIJX9DK2EBEwokRM9kDwjBVtag,ChIJmZKM98dgwokRZ6ZvctE2ImQ,ChIJ3wsoxKFZwokRRg8OvEjE0bc,ChIJQRpR6Jj0wokRl9myJ19eEvU,ChIJQYmJtx5awokR45_sopkXVNE,ChIJh4MG8h9awokR1kDLslu2C3E,ChIJbdnGmlxewokRobPWHJXQVjM
id,,,,,,,,,,,,,,,,,,,,,
Depot,0.908164,3.624313,3.06439,4.389933,10.402506,7.179099,0.970495,5.890938,17.795796,17.759441,...,13.875805,8.577781,5.44481,15.372453,11.601055,2.636302,13.965268,6.172048,6.172048,6.126072
ChIJwTC7A55ZwokRPNX9g7ngbaI,4.487928,0.69232,1.275166,3.404734,11.91588,6.139603,1.165866,6.603732,17.565902,17.655366,...,11.143219,8.364849,5.315607,15.963847,11.432615,0.907903,11.489765,1.901283,1.901283,7.48685
ChIJU1XImaNZwokRutunetC8XeE,5.226573,1.238316,0.69339,4.251927,12.064494,5.551015,1.546753,5.13778,17.167576,18.68499,...,13.796589,7.155312,5.823538,12.936463,11.820069,0.758407,14.100627,2.324565,2.324565,8.534553
ChIJG7L-TLVbwokRT36uIrwz2Mo,4.35525,2.95688,4.039189,0.696478,9.573396,10.271436,2.541625,11.223,16.794486,4.019016,...,8.569997,11.704933,2.211792,11.801273,12.724804,4.125083,14.666276,2.171729,2.171729,6.119356
ChIJ5cPkuBtgwokRn55JgpGqjFA,13.477798,12.874358,10.590967,9.80648,0.996817,10.636402,11.766712,11.223698,9.554601,18.6048,...,6.510232,11.005756,6.830633,6.263304,8.757591,13.914388,12.551039,13.131675,13.131675,12.911588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJ3wsoxKFZwokRRg8OvEjE0bc,4.79565,0.923201,0.723722,4.286476,11.202902,5.612903,1.651204,5.459457,16.934039,17.158424,...,11.603017,7.469938,6.504683,14.693145,12.141539,0.67782,13.880918,2.55509,2.55509,8.124578
ChIJQRpR6Jj0wokRl9myJ19eEvU,13.462167,7.422576,10.747481,11.828408,7.453325,7.339817,5.455323,5.742348,20.610216,15.2434,...,12.606605,4.035693,9.354276,13.006796,13.507891,11.292802,1.349578,11.045547,11.045547,6.673253
ChIJQYmJtx5awokR45_sopkXVNE,5.905736,2.135587,2.268411,2.207256,13.13874,9.151589,0.889933,6.893867,18.390186,18.863063,...,10.674825,9.968347,3.503519,12.979222,12.969834,2.693479,13.744055,0.726178,0.726178,9.72825


In [46]:
# Export the DataFrame to a CSV file
costMatrixFinancial.to_csv('costMatrixFinancial.csv')